<a href="https://colab.research.google.com/github/paradisepilot/study/blob/master/earth-observation/earth-engine/e2e-gee/e2e-gee-06-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>